**Objectifs du travail**: Le but de ce travail pratique est de prédire le mot manquant dans un proverbe incomplet en utilisant un transformer. Il a aussi pour but d'évoluer la performance d'un transformer, en particulier Bert dans notre cas, avant et après fine-tuning.

**Installation et déclaration des variables**
Pour la bonne exécution du travail, certaines dépendances sont installées.
Pour ce travail, nous avons sélectionné le model bert-base-multilingual-uncased de Hugging face. Ce choix se justifie par le fait que notre corpus est en français.

In [2]:
!pip install transformers
!pip install transformers[sentencepiece]
!pip install sacremoses
from transformers import BertTokenizer, BertForMaskedLM, TrainingArguments, Trainer
from transformers import pipeline
import torch
import json
import numpy as np
cuda_device = 1
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_checkpoint = "bert-base-multilingual-uncased"
model = BertForMaskedLM.from_pretrained(model_checkpoint)
model = model.to(device)
tokenizer = BertTokenizer.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/641M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/851k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

**Fonctions utilitaires**:
Les fonctions ***load_proverbs*** et ***load_tests*** servent à lire et récupérer respectivement les données pour l'entrainement et pour le test.
La fonction ***score*** est utilisée pour calculer la perplexité d'un provebe test. Cette fonction utilise le tokeniseur pour encoder le proverbe. Après celà, chaque mot du proverbe est remplacé par le masque du modèle de manière itérative puis la perplexité est calculée. La perplexité du proverbe est alors assimilée à la somme de la perplexité de chaque mot.

La fonction ***get_solutions*** permet de récupérer les solutions i.e. les vrais mots à trouver.

La fonction ***get_predictions*** utilise la fonction ***score*** pour déterminer l'option la plus plausible. Pour ce faire, pour chaque proverbe incomplet, les 3 astérix sont remplacés par une option. Puis la perplexité du proverbe "complété" est calculée. L'option du proverbe avec la plus faible perplexité est considérée comme la solution la plus plausible. 

La fonction ***precision_metric*** donne la précision du modèle après la prédiction. Elle calcule le taux des proverbes bien prédits

In [3]:
# Method to load train data from file
def load_proverbs(filename):
    with open(filename, 'r', encoding='utf-8') as f:
        raw_lines = f.readlines()
    return [x.strip() for x in raw_lines]

# Method to load test data from file
def load_tests(filename):
    with open(filename, 'r', encoding='utf-8') as fp:
        test_data = json.load(fp)
    return test_data

def score(model, tokenizer, sentence):
    tensor_input = tokenizer.encode(sentence, return_tensors='pt').to(device)
    repeat_input = tensor_input.repeat(tensor_input.size(-1)-2, 1).to(device)
    mask = torch.ones(tensor_input.size(-1) - 1).diag(1)[:-2].to(device)
    masked_input = repeat_input.masked_fill(mask == 1, tokenizer.mask_token_id).to(device)
    labels = repeat_input.masked_fill( masked_input != tokenizer.mask_token_id, -100).to(device)
    with torch.inference_mode():
        loss = model(masked_input, labels=labels).loss
    return np.exp(loss.item())

def get_solutions(test_data):
    return [data["solution"] for data in test_data]

def get_predictions(test_data, model, tokenizer):
    predictions = []
    for datas in test_data:
        scores = {}
        partial_proverb = datas['test']
        for option in datas['choices']:
            incomplet_proverb = partial_proverb.replace("***", option)
            scores[option] = score(model, tokenizer, incomplet_proverb)
        good_sentence = min(scores, key=scores.get)
        predictions.append(good_sentence)
        scores.clear()

    return predictions

def precision_metric(y_true, y_pred):
    true_pred = 0

    for idx in range(len(y_pred)):
        if y_true[idx] == y_pred[idx]:
            true_pred += 1
    
    return true_pred * 100 / len(y_pred)

**1. Sans fine-tuning**

In [4]:
tests_proverbs = load_tests("data/test_proverbes.json")
word_predictions = get_predictions(tests_proverbs, model, tokenizer)
right_words = get_solutions(tests_proverbs)
print(f"La précision avant fine tuning est : {precision_metric(right_words, word_predictions)}")

La précision avant fine tuning est : 35.714285714285715


**2. Fine tuning**

La fonction ***prepare_data*** permet de préparer les données d'entrainement comme suit:
* Tokenisation : le tokenizer de Bert ***BertTokenizer*** est utilisé pour tokeniser chaque proverbe du corpus. La taille des tokens est est fixée à 32. Dans le cas où la taille de tokens d'un proverb est supérieure à la taille limite, le vecteur de tokens est tronqué d'ouù le paramètre ***truncation=true***. Dans le cas où la taille du vecteur de tokens est inférieure alors un padding est appliqué pour atteindre la taille de 32.
* Création des labels : au vecteur obtenu après la tokénisation, une colonne est ajoutée pour les labels. Ses valeurs sont une copie de des propriétés ***inputs_ids*** 
* Masquage : Nous avons appliqué la théorie de Bert qui consiste à masquer 15% des tokens du jeu de données. Le masquage consiste à remplacé un token d'un proverbe par ce token spécifique, MASK. Le masquage peut être appliqué à tout token dont l'index est différent de 101, 102. En effet, lors de la tokénisation, des tokens spécifiques (CLS et SEP) ont été ajoutés et les index 101 et 102 leur ont été attribués respectivement. L'index 103 est attribué au token MASK

Les données obtenues après la préparation sont utilisées pour entrainer (ajuster) le modèle avec notre corpus. Nous avons utilisé Trainer et TrainingArguments de Hugging Face

In [5]:
proverbs = load_proverbs("data/proverbes.txt")

def prepare_data(data_train):
    inputs_train = tokenizer(data_train, return_tensors='pt', max_length=32, truncation=True, padding='max_length')
    inputs_train['labels'] = inputs_train.input_ids.detach().clone()
    # Create and add mask
    rand_float = torch.rand(inputs_train.input_ids.shape)
    mask_arr = (rand_float < 0.15) * (inputs_train.input_ids != 101) * (inputs_train.input_ids != 102) * (inputs_train.input_ids != 0)
    selection = []

    for i in range(inputs_train.input_ids.shape[0]):
        selection.append(
            torch.flatten(mask_arr[i].nonzero()).tolist()
        )

    for i in range(inputs_train.input_ids.shape[0]):
        inputs_train.input_ids[i, selection[i]] = 103
        
    return inputs_train

inputs_train = prepare_data(proverbs)

In [6]:
class LoaderDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    def __len__(self):
        return len(self.encodings.input_ids)

    #Entrainement du modèle avec notre jeu de données.
datasets = LoaderDataset(inputs_train)
args = TrainingArguments(
    report_to='none',
    output_dir='out',
    overwrite_output_dir=True,
    per_device_train_batch_size=16,
    num_train_epochs=10,
    prediction_loss_only=False,
    do_train=True,
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=datasets,
)

trainer.train()

/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 3108
  Num Epochs = 10
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 1950
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """


Step,Training Loss
500,0.240300
1000,0.026700
1500,0.003900


Saving model checkpoint to out/checkpoint-500
Configuration saved in out/checkpoint-500/config.json
Model weights saved in out/checkpoint-500/pytorch_model.bin
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """
Saving model checkpoint to out/checkpoint-1000
Configuration saved in out/checkpoint-1000/config.json
Model weights saved in out/checkpoint-1000/pytorch_model.bin
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """
Saving model checkpoint to out/checkpoint-1500
Configuration saved in out/checkpoint-1500/config.json
Model weights saved in out/checkpoint-1500/

TrainOutput(global_step=1950, training_loss=0.06975801174457257, metrics={'train_runtime': 286.6873, 'train_samples_per_second': 108.411, 'train_steps_per_second': 6.802, 'total_flos': 511725031211520.0, 'train_loss': 0.06975801174457257, 'epoch': 10.0})

In [7]:
tests_proverbs = load_tests("data/test_proverbes.json")
word_predictions = get_predictions(tests_proverbs, model, tokenizer)
right_words = get_solutions(tests_proverbs)
print(f"La précision après fine tuning est : {precision_metric(right_words, word_predictions)}")

La précision après fine tuning est : 53.57142857142857


**3. Discussions**
Avant le fine-tuning, le modèle a une précision de 35%. Cependant, après le fine-tuning, la précision est à 55%. Cela peut s'expliquer par le fait que sans le fine-tuning, le modèle peut rencontrer des tokens du jeu de test dont il n'avait pas vu lors de l'entrainement. De ce fait, lors de la prédiction d'un mot, le modèle lui attribuera un label pouvant être erroné. Pour éviter ce problème, il n'est nécessaire de faire du fine-tuning. Ce dernier permet d'entrainer le modèle avec notre corpus ce qui permettrait au modèle de faire le moins d'erreur lors du test.
En conclusion, nous pouvons dire que le fine-tuning permet un réajustement du modèle pour un meilleur apprentissage du corpus et par conséquent une meilleure prédiction.
En perspective, il serait intéressant de faire un fine-tuning sur les différentes composantes (les couches) du modèle pour explorer leurs influences sur la performance de BERT.